# Setup

In [ ]:
!pip install scipy==1.6.3
!pip install scikit_optimize==0.8.1
!pip install scikit_learn==0.24.2
! git clone https://github.com/zykhoo/predicting_hamiltonian_dynamics.git

     |████████████████████████████████| 27.4 MB 2.0 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 101 kB 3.9 MB/s 
     |████████████████████████████████| 22.3 MB 1.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but y

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run all

In [ ]:
import numpy as np

experiment,sys,dim = "NN","morsepotential",2
D=10
alpha=0.5
m=1.
f2 = lambda z: -2*D*alpha*(np.exp(-alpha*z[0])-np.exp(-2*alpha*z[0]))
f1 = lambda x: x[1]
f = lambda z: -2*D*alpha*(np.exp(-alpha*z)-np.exp(-2*alpha*z))
H = lambda z: 1/2*np.asarray(z[1])**2 + D*(1-np.exp(-alpha*z[0]))**2
spacedim=[(-1., 4.), (-3.0,3.0)]
fvector = lambda x: np.asarray([f1(x), f2(x)])
h= 0.1
x0, H0 = 0.,0.
initialcon = [64, 128, 256, 512, 1024]
LR=0.001
diagdist = np.sum(np.square(np.asarray([spacedim[0][0], spacedim[0][1]]), np.asarray([spacedim[1][0], spacedim[1][1]])))
epsilon = 0.1

from predicting_hamiltonian_dynamics import groundtruth_2dim
import numpy as np 

"""# Test dataset creation"""

from tqdm import tqdm
import time 

evaluation_length_long = 50
evaluation_length_one = 1

xxlong,yylong = np.linspace(spacedim[0][0], spacedim[0][1], 5), np.linspace(spacedim[1][0], spacedim[1][1], 5)
xlong,ylong = np.meshgrid(xxlong,yylong)
xxshort,yyshort = np.linspace(spacedim[0][0], spacedim[0][1], 20), np.linspace(spacedim[1][0], spacedim[1][1], 20)
xshort,yshort = np.meshgrid(xxshort,yyshort)

trajectories_groundtruth_start_long = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xlong),np.ravel(ylong)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
  trajectories_groundtruth_start_long = np.vstack((trajectories_groundtruth_start_long, results_200))
trajectories_groundtruth_start_long.shape # output no., start and final, p and q, full traj

trajectories_groundtruth_start_short = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xshort),np.ravel(yshort)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
  trajectories_groundtruth_start_short = np.vstack((trajectories_groundtruth_start_short, results_200))
trajectories_groundtruth_start_short.shape # output no., start and final, p and q, full traj

within_array = groundtruth_2dim.get_within_array(trajectories_groundtruth_start_long, spacedim)

100%|██████████| 400/400 [00:00<00:00, 4018.07it/s]


In [ ]:
path = '/content/drive/MyDrive/SSI/Baseline v2/synthetic systems (upload)/'

In [ ]:
from predicting_hamiltonian_dynamics.models import NN_2dim
from predicting_hamiltonian_dynamics.models import GP_2dim
from predicting_hamiltonian_dynamics.models import PINN_2dim
from predicting_hamiltonian_dynamics.models import PIGP_2dim


for i in range(20):
  seed = i
  np.random.seed(seed=seed)
  for ini in initialcon: 

    start, final = groundtruth_2dim.CreateTrainingDataTrajClassicIntRandom(1,ini,spacedim,h,f1,f2,seed = seed,n_h = 1)

    delta = start.copy()
    delta[0,:] = f1(start)
    delta[1,:] = f2(start)

    """# NN"""

    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


    wholemat, evalmat = NN_2dim.data_preprocessing(start, delta, device)

    import torch.optim as optim
    import time 

    net = NN_2dim.Net(dim,16,dim)
    starttime = time.time() 
    net = NN_2dim.train(net, wholemat, evalmat, optimizer=optim.Adam(net.parameters(), lr=LR), batchsize=10, iter=1600, )
    traintime = time.time()-starttime

    from tqdm import tqdm
    from predicting_hamiltonian_dynamics import metrics


    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = NN_2dim.compute_metrics_NN(net, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'MorsePotential_random.txt', 'a')
    file_object.write('NN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# GP"""

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF
    from sklearn import preprocessing

    output = delta
    scaler = preprocessing.MinMaxScaler()
    input = scaler.fit_transform(start.transpose())
    kernel = 1 * RBF(length_scale=2., length_scale_bounds="fixed")
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, optimizer = None, random_state = seed)
    starttime = time.time() 
    gaussian_process.fit(input, output.transpose())
    traintime = time.time()-starttime
    gaussian_process.predict(scaler.transform([[0.7,0.8]]))

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = GP_2dim.compute_metrics_GP(gaussian_process, scaler, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'MorsePotential_random.txt', 'a')
    file_object.write('GP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# PINN"""

    from predicting_hamiltonian_dynamics.models import PINN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    wholemat, evalmat = PINN_2dim.data_preprocessing(start, delta,device)
    net=PINN_2dim.Net(dim,16,1)
    starttime = time.time()
    net,avg_vallosses,avg_lossli,avg_f1li,avg_f2li,avg_f3li,avg_f4li=PINN_2dim.train(net,bs=10,num_epoch=5000,initial_conditions=initialcon,device=device,
                                                                                    wholemat=wholemat,evalmat=evalmat,x0=x0,H0=H0,dim=dim,LR=LR,patience=500,c1=1,c2=1,c3=1,c4=1)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PINN_2dim.compute_metrics_PINN(net, device, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)
    
    file_object = open(path + 'MorsePotential_random.txt', 'a')
    file_object.write('PINN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()

    """# PIGP"""

    from predicting_hamiltonian_dynamics.models import PIGP_2dim
    GP = PIGP_2dim.BertalanGP()
    starttime = time.time()
    GP.train(start,delta,h)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PIGP_2dim.compute_metrics_PIGP(GP, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'MorsePotential_random.txt', 'a')
    file_object.write('PIGP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


training loss tensor(7.8526, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(6.0280, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(2.7411, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(3.6085, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.8574, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(1.3811, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0809, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0938, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0405, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0369, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0259, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0254, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0174, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation lo

  0%|          | 0/50 [00:00<?, ?it/s]/content/predicting_hamiltonian_dynamics/models/NN_2dim.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fstage = lambda stg: h * torch.squeeze(net(torch.tensor([q+stg,p]).float().transpose(1,0)),0).detach().numpy().transpose()[:dim]
100%|██████████| 50/50 [00:00<00:00, 190.53it/s]

100%|██████████| 50/50 [00:00<00:00, 724.89it/s]

100%|██████████| 50/50 [00:00<00:00, 230.20it/s]

100%|██████████| 50/50 [00:00<00:00, 164.55it/s]

100%|██████████| 50/50 [00:00<00:00, 192.12it/s]

100%|██████████| 50/50 [00:00<00:00, 876.64it/s]

100%|██████████| 5/5 [00:00<00:00, 247.76it/s]

100%|██████████| 50/50 [00:00<00:00, 99.86it/s] 

100%|██████████| 50/50 [00:00<00:00, 100.24it/s]

100%|██████████| 50/50 [00:00<00:00, 130.04it/s]

100%|██████

 
epoch= 0  time= 0.023472309112548828  loss= 12.857238133748373  val_loss= 17.411478757858276  f1= 2.497340375185401  f2= 10.337411951169392  f3= 0.002240144550645103  f4= 0.020245629117528554 percent lr= 0.001
 
epoch= 10  time= 0.2002718448638916  loss= 12.40812603632609  val_loss= 16.79072856903076  f1= 2.674427744606684  f2= 9.69964665382403  f3= 0.006499954887355368  f4= 0.027551849140261944 percent lr= 0.001
 
epoch= 20  time= 0.3753023147583008  loss= 12.505642493565878  val_loss= 15.726126432418823  f1= 2.86177786294181  f2= 9.455963984279528  f3= 0.016801368445158005  f4= 0.17109928029005014 percent lr= 0.001
 
epoch= 30  time= 0.5510668754577637  loss= 10.944135109583536  val_loss= 14.646244525909424  f1= 1.7570009269777656  f2= 8.912035842691301  f3= 0.019654002351065476  f4= 0.25544429174174865 percent lr= 0.001
 
epoch= 40  time= 0.7418467998504639  loss= 10.850991090138754  val_loss= 13.6753751039505  f1= 1.9094239909288742  f2= 8.2018487471383  f3= 0.012082514663537344 

100%|██████████| 400/400 [00:02<00:00, 138.81it/s]


Start training with for 64 data points
Start computation of covariance matrix.
Covariance matrix of shape (64, 64)computed.
Start Cholesky decomposition of (64, 64) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (129, 64)


...

100%|██████████| 50/50 [00:00<00:00, 283.87it/s]

100%|██████████| 50/50 [00:00<00:00, 1228.73it/s]

100%|██████████| 50/50 [00:00<00:00, 261.88it/s]

100%|██████████| 50/50 [00:00<00:00, 148.79it/s]

100%|██████████| 50/50 [00:00<00:00, 183.07it/s]

100%|██████████| 50/50 [00:00<00:00, 1766.34it/s]

100%|██████████| 5/5 [00:00<00:00, 84.13it/s]

100%|██████████| 50/50 [00:00<00:00, 284.10it/s]

100%|██████████| 50/50 [00:00<00:00, 171.49it/s]

100%|██████████| 50/50 [00:00<00:00, 166.07it/s]

100%|██████████| 50/50 [00:00<00:00, 125.09it/s]

100%|██████████| 50/50 [00:00<00:00, 174.24it/s]

100%|██████████| 50/50 [00:00<00:00, 1723.13it/s]

100%|██████████| 50/50 [00:00<00:00, 177.55it/s]

100%|██████████| 50/50 [00:00<00:00, 270.66it/s]

100%|██████████| 50/50 [00:00<00:00, 1543.69it/s]

100%|██████████| 50/50 [00:00<00:00, 249.37it/s]

100%|██████████| 50/50 [00:00<00:00, 183.34it/s]

100%|██████████| 50/50 [00:00<00:00, 1935.62it/s]

100%|██████████| 6/6 [00:00<00:00, 391.66it/s]



 
epoch= 0  time= 0.28955769538879395  loss= 10.784366357617262  val_loss= 10.572851157188415  f1= 2.703250659367296  f2= 8.011258482472197  f3= 0.03302972499144877  f4= 0.03682743368723239 percent lr= 0.001
 
epoch= 10  time= 3.352369546890259  loss= 1.4290669625852166  val_loss= 1.1317662924528122  f1= 0.1365453843169462  f2= 0.9066888010463843  f3= 0.0019399829442724096  f4= 0.383892772037921 percent lr= 0.001
 
epoch= 20  time= 6.377989053726196  loss= 0.18409955937687944  val_loss= 0.17330677919089793  f1= 0.030046333357023405  f2= 0.10650115284955347  f3= 0.0004353583855361919  f4= 0.04711671390045946 percent lr= 0.001
 
epoch= 30  time= 9.438332080841064  loss= 0.06086717974167408  val_loss= 0.06951537313871085  f1= 0.008431546405477466  f2= 0.03079407887455062  f3= 0.0005411447377089022  f4= 0.021100410013432064 percent lr= 0.001
 
epoch= 40  time= 12.50359296798706  loss= 0.026206684026231126  val_loss= 0.035819007258396596  f1= 0.0027903125505313563  f2= 0.010636682547322161 

100%|██████████| 400/400 [00:02<00:00, 182.22it/s]


Start training with for 1024 data points
Start computation of covariance matrix.
Covariance matrix of shape (1024, 1024)computed.
Start Cholesky decomposition of (1024, 1024) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (2049, 1024)


100%|██████████| 400/400 [09:44<00:00,  1.46s/it]
